In [ ]:
2+2

: 

In [ ]:
!pip3 install torchmetrics==0.4.1
!pip3 install transformers==4.8.2
!pip3 install pytorch_lightning==1.3.8

: 

In [2]:
from transformers import T5ForConditionalGeneration, AutoTokenizer

model = T5ForConditionalGeneration.from_pretrained('jenspt/byt5_extra_layer_1024_ft_all_clean_data_SAFETY_v2')
tokenizer = AutoTokenizer.from_pretrained('google/byt5-small')

Downloading:   0%|          | 0.00/683 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

Some weights of the model checkpoint at jenspt/byt5_extra_layer_1024_ft_all_clean_data_SAFETY_v2 were not used when initializing T5ForConditionalGeneration: ['lm_head.0.bias', 'lm_head.2.bias', 'lm_head.0.weight', 'lm_head.2.weight']
- This IS expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of T5ForConditionalGeneration were not initialized from the model checkpoint at jenspt/byt5_extra_layer_1024_ft_all_clean_data_SAFETY_v2 and are newly initialized: ['decoder.final_layer_norm.weight']
You should probably TRAIN this model on a down-stream task to be a

Downloading:   0%|          | 0.00/2.59k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/698 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.50k [00:00<?, ?B/s]

In [3]:
import sys
from google.colab import drive
drive.mount('/content/drive')
drive_path = 'drive/My Drive/projekt/'

sys.path.append(drive_path) 

Mounted at /content/drive


In [4]:
import pandas as pd
df = pd.read_pickle(drive_path+'clean_data_test.pkl')

df.head()

,wer,cer,reference_text,transcription,book_name,test_wer,ref_words,num_words
89961,0.357143,0.066667,rufus gifford om sit bryllup med stephen devin...,rufus gifford om sit bryllup med stephen de vi...,Ambassadøren,0.312500,"[rufus, gifford, om, sit, bryllup, med, stephe...",14
29664,0.400000,0.030303,afbryd jobbet og vis hjælpelinien,afbryd jobbet og vis hjælpe linjen,nst test,0.333333,"[afbryd, jobbet, og, vis, hjælpelinien]",5
34713,0.000000,0.000000,mine forældre var begyndt at trække på smilebå...,mine forældre var begyndt at trække på smilebå...,Den sønderjyske farm,0.000000,"[mine, forældre, var, begyndt, at, trække, på,...",25
110588,0.052632,0.024096,lad os aftale at du tager med og hvis der komm...,lad os aftale du tager med og hvis der kommer ...,"Mørke, dybe skove",0.055556,"[lad, os, aftale, at, du, tager, med, og, hvis...",19
72325,0.000000,0.000000,de siger for meget nej og for lidt ja,de siger for meget nej og for lidt ja,Personlig karma,0.000000,"[de, siger, for, meget, nej, og, for, lidt, ja]",9


In [5]:
X = df['transcription'].tolist() 
y = df['reference_text'].tolist()

In [6]:
from torch.utils.data import DataLoader
test_inputs = tokenizer(X, return_tensors="pt", padding=True, truncation=True, max_length=256)
test_tgt = tokenizer(y, return_tensors="pt", padding=True, truncation=True, max_length=256)
#test_inputs['labels'] = test_tgt['input_ids']

In [8]:
import torch

class TextDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, y_encodings=None):
        self.encodings = encodings

    def __getitem__(self, idx):
        item= {key: val[idx] for key, val in self.encodings.items()}
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])

test_dataset = TextDataset(test_inputs)

def get_train_dataloader(dataset):
    return DataLoader(
        dataset, batch_size=8, shuffle=False, drop_last=True,
        num_workers=0
    )

dataloader = get_train_dataloader(test_dataset)

In [9]:
from pytorch_lightning.utilities.apply_func import move_data_to_device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Running on: {}".format(device))
model = model.to(device)
model.eval()

out = []
      
for i, batch in enumerate(dataloader):
  if i % 50 ==0:
    print(i/len(dataloader))
  batch = move_data_to_device(batch, device)
  output = model.generate(
              input_ids=batch["input_ids"], attention_mask=batch["attention_mask"],
              repetition_penalty=1.0, length_penalty=1.0, max_length=256,
              num_beams=1, num_return_sequences=1,
              output_scores=True, return_dict_in_generate=True
          )
    
  outputs = tokenizer.batch_decode(output.sequences, skip_special_tokens=True)
  out.append(outputs)

Running on: cuda:0
0.0
0.0390015600624025
0.078003120124805
0.11700468018720749
0.15600624024961
0.19500780031201248
0.23400936037441497
0.27301092043681746
0.31201248049922
0.35101404056162244
0.39001560062402496
0.4290171606864275
0.46801872074882994
0.5070202808112324
0.5460218408736349
0.5850234009360374
0.62402496099844
0.6630265210608425
0.7020280811232449
0.7410296411856474
0.7800312012480499
0.8190327613104524
0.858034321372855
0.8970358814352574
0.9360374414976599
0.9750390015600624


In [10]:
def flatten(t):
    return [item for sublist in t for item in sublist]

out = flatten(out)

In [11]:
out.append("NAN")
out.append("NAN")

In [12]:
df['corrected'] = out

In [14]:
df_clean = df.iloc[:-2,:] #We have to remove the last two rows because I ran the test with "drop_last =True"

In [15]:
df_clean.to_pickle(drive_path + "Gemte outputs/ByT5_extra_layer.pkl")

In [18]:
df_clean.iloc[160:190,[0,1,2,3,8]]

,wer,cer,reference_text,transcription,corrected
46062,0.000000,0.000000,man kan godt forstå sagde min far og lagde arm...,man kan godt forstå sagde min far og lagde arm...,...
2717,0.076923,0.026667,vi forhandler os til en overtagelse uden chauf...,vi forhandler os til en overtagelse uden chauf...,yدa...
35436,0.000000,0.000000,meget meget meget,meget meget meget,aaaaN...
32490,0.000000,0.000000,masser af mennesker lever af at repræsentere f...,masser af mennesker lever af at repræsentere f...,aa...
60471,0.100000,0.052632,du tror måske at du får en gave af mig,du tror måske du får en gave af mig,aؔaa...
3539,0.000000,0.000000,det vilde liv var det eneste jeg kunne ty til,det vilde liv var det eneste jeg kunne ty til,a...
137257,0.222222,0.000000,jeg har en smule sparepenge jeg ordner det hele,jeg har en smule spare penge jeg ordner det hele,$aaۀa...
8320,0.000000,0.000000,spurgte han lige ud i luften,spurgte han lige ud i luften,$...
12669,0.000000,0.000000,dyrets opmærksomhed var heldigvis rettet mod n...,dyrets opmærksomhed var heldigvis rettet mod n...,a$6aa...
41103,0.000000,0.000000,der skete en teknisk fejl men det var efter læ...,der skete en teknisk fejl men det var efter læ...,a$aaaa...
